In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from keras.layers.core import Activation
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, LSTM, Activation
from keras.utils.generic_utils import get_custom_objects
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.callbacks import ModelCheckpoint

time: 1.14 s (started: 2022-07-22 04:31:52 +00:00)


In [2]:
# Train - Dev - Test Generation
train_raw = pd.read_csv('./dataset/train.csv').drop('Unnamed: 0', axis=1)
dev_raw = pd.read_csv('./dataset/dev.csv').drop('Unnamed: 0', axis=1)
test1_raw = pd.read_csv('./dataset/test1.csv').drop('Unnamed: 0', axis=1)
test2_raw = pd.read_csv('./dataset/test2.csv').drop('Unnamed: 0', axis=1)

train_X = train_raw.iloc[:, :-1]
dev_X = dev_raw.iloc[:, :-1]
test1_X = test1_raw.iloc[:, :-1]
test2_X = test2_raw.iloc[:, :-1]
train_Y = train_raw.iloc[:, -1]
dev_Y = dev_raw.iloc[:, -1]
test1_Y = test1_raw.iloc[:, -1]
test2_Y = test2_raw.iloc[:, -1]


# data sampling
STEP = 20
#num_list = [STEP*i for i in range(int(1117500/STEP))]

_train_X = np.asarray(train_X).reshape((int(1117500/STEP), 20, 1))
_dev_X = np.asarray(dev_X).reshape((int(1117500/STEP), 20, 1))
_test1_X = np.asarray(test1_X).reshape((int(1117500/STEP), 20, 1))
_test2_X = np.asarray(test2_X).reshape((int(1117500/STEP), 20, 1))

_train_Y = np.asarray(train_Y).reshape(int(1117500/STEP), 1)
_dev_Y = np.asarray(dev_Y).reshape(int(1117500/STEP), 1)
_test1_Y = np.asarray(test1_Y).reshape(int(1117500/STEP), 1)
_test2_Y = np.asarray(test2_Y).reshape(int(1117500/STEP), 1)

time: 691 ms (started: 2022-07-22 04:31:54 +00:00)


In [3]:
def double_tanh(x):
    return (K.tanh(x) * 2)

#get_custom_objects().update({'double_tanh':Activation(double_tanh)})

# Model Generation
model = Sequential()
#check https://machinelearningmastery.com/use-weight-regularization-lstm-networks-time-series-forecasting/
model.add(LSTM(25, input_shape=(20,1), dropout=0.0, kernel_regularizer=l1_l2(0.00,0.00), bias_regularizer=l1_l2(0.00,0.00)))
model.add(Dense(1, activation=double_tanh))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse', 'mae'])
#, kernel_regularizer=l1_l2(0,0.1), bias_regularizer=l1_l2(0,0.1),

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 25)                2700      
                                                                 
 dense (Dense)               (None, 1)                 26        
                                                                 
Total params: 2,726
Trainable params: 2,726
Non-trainable params: 0
_________________________________________________________________
time: 622 ms (started: 2022-07-22 04:31:54 +00:00)


2022-07-22 04:31:54.773658: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-22 04:31:54.776323: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-22 04:31:54.776451: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-22 04:31:54.776712: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [4]:
# Fitting the Model
model_scores = {}
Reg = False
d = 'LSTM_only_new_api'

if Reg :
    d += '_with_reg'

epoch_num=1
max_epoch = 3500
for _ in range(max_epoch):

    # train the model
    dir_ = './lstm_only_models/'+d
    file_list = os.listdir(dir_)
    if len(file_list) != 0 :
        epoch_num = len(file_list) + 1
        recent_model_name = 'epoch'+str(epoch_num-1)
        filepath = './lstm_only_models/' + d + '/' + recent_model_name
        # custom_objects = {"double_tanh": double_tanh}
        # with keras.utils.custom_object_scope(custom_objects):
        model = load_model(filepath)

    filepath = './lstm_only_models/' + d + '/epoch'+str(epoch_num)

    # checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=False, mode='min')
    model.fit(_train_X, _train_Y, epochs=1, batch_size=500, shuffle=True)
    model.save(filepath)
    
    #callbacks_list = [checkpoint]
    #if len(callbacks_list) == 0:
    #    model.fit(_train_X, _train_Y, epochs=1, batch_size=500, shuffle=True)
    #else:
    #    model.fit(_train_X, _train_Y, epochs=1, batch_size=500, shuffle=True, callbacks=callbacks_list)

    # test the model
    score_train = model.evaluate(_train_X, _train_Y)
    score_dev = model.evaluate(_dev_X, _dev_Y)
    score_test1 = model.evaluate(_test1_X, _test1_Y)
    score_test2 = model.evaluate(_test2_X, _test2_Y)

    print('train set score : mse - ' + str(score_train[1]) +' / mae - ' + str(score_train[2]))
    print('dev set score : mse - ' + str(score_dev[1]) +' / mae - ' + str(score_dev[2]))
    print('test1 set score : mse - ' + str(score_test1[1]) +' / mae - ' + str(score_test1[2]))
    print('test2 set score : mse - ' + str(score_test2[1]) +' / mae - ' + str(score_test2[2]))
#.history['mean_squared_error'][0]
    # get former score data
    df = pd.read_csv("./lstm_only_scores/"+d+".csv")
    train_mse = list(df['TRAIN_MSE'])
    dev_mse = list(df['DEV_MSE'])
    test1_mse = list(df['TEST1_MSE'])
    test2_mse = list(df['TEST2_MSE'])

    train_mae = list(df['TRAIN_MAE'])
    dev_mae = list(df['DEV_MAE'])
    test1_mae = list(df['TEST1_MAE'])
    test2_mae = list(df['TEST2_MAE'])

    # append new data
    train_mse.append(score_train[1])
    dev_mse.append(score_dev[1])
    test1_mse.append(score_test1[1])
    test2_mse.append(score_test2[1])

    train_mae.append(score_train[2])
    dev_mae.append(score_dev[2])
    test1_mae.append(score_test1[2])
    test2_mae.append(score_test2[2])

    # organize newly created score dataset
    model_scores['TRAIN_MSE'] = train_mse
    model_scores['DEV_MSE'] = dev_mse
    model_scores['TEST1_MSE'] = test1_mse
    model_scores['TEST2_MSE'] = test2_mse

    model_scores['TRAIN_MAE'] = train_mae
    model_scores['DEV_MAE'] = dev_mae
    model_scores['TEST1_MAE'] = test1_mae
    model_scores['TEST2_MAE'] = test2_mae
    
    # save newly created score dataset
    model_scores_df = pd.DataFrame(model_scores)
    model_scores_df.to_csv("./lstm_only_scores/"+d+".csv")

INFO:tensorflow:Assets written to: ./lstm_only_models/LSTM_only_new_api/epoch1/assets


INFO:tensorflow:Assets written to: ./lstm_only_models/LSTM_only_new_api/epoch1/assets
2022-07-22 04:31:58.646050: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


 34/112 [========>.....................] - ETA: 0s - loss: 0.2262 - mse: 0.2262 - mae: 0.3973 

2022-07-22 04:31:59.069066: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1747/1747 [==============================] - 3s 2ms/step - loss: 0.3282 - mse: 0.3282 - mae: 0.4672
train set score : mse - 0.1898219734430313 / mae - 0.3627184331417084
dev set score : mse - 0.284330278635025 / mae - 0.4344899654388428
test1 set score : mse - 0.2777898907661438 / mae - 0.4310569167137146
test2 set score : mse - 0.32823866605758667 / mae - 0.46721434593200684


...

INFO:tensorflow:Assets written to: ./lstm_only_models/LSTM_only_new_api/epoch3/assets


INFO:tensorflow:Assets written to: ./lstm_only_models/LSTM_only_new_api/epoch3/assets


 232/1747 [==>...........................] - ETA: 2s - loss: 0.2579 - mse: 0.2579 - mae: 0.4118

KeyboardInterrupt: 

time: 45.4 s (started: 2022-07-22 04:31:55 +00:00)


In [ ]:
get